![Astrofisica Computacional](logo.png)

---
## Introduction to Lattice-Boltzmann (LB) Methods

Eduard Larrañaga (ealarranaga@unal.edu.co)

---

---

## Solving the Hydrodynamics Equations

Many of the fluid simulation methods use a discretization of a 3-dimensional partial differential equation representing motion in a continuum. However, other approaches, such as the Lattice-Boltzmann (LB) method,  considers fluids from a statistical mechanics point of view, treating them as a set of particles with probability distributions on their locations and velocities and simulating the evolution of these distributions.

## Boltzmann Transport Equation

Now, we will introduce the Boltzmann transport equation. However, we will ignore many physical aspects, focusing solely on the mathematics and nature of the differential equation. For example, in order to obtain physically accurate simulations, it is required to pay attention to units and scaling constants but here, we will ignore these considerations. Detailed presentations can be consulted elsewhere. 

The fundamental assumption behind the Boltzmann equation is that we can describe a system as a probability distribution over particles. Each particle has a probability density $f(\textbf{x},\textbf{v},t)$ associated with it, which represents the probability that the particle is at position $x$ with velocity $v$ at time $t$. From this assumption, we can derive a collision-less variant of the Boltzmann transport equation.

Note that given the probability distribution $f$, the macroscopic quantities of interest by integrating this function. For example,
$$
\begin{align}
\rho (\textbf{x},t) = &\int f(\textbf{x},\textbf{v},t) d\textbf{v}\\
\rho (\textbf{x},t) \textbf{v} = &\int \textbf{v} f(\textbf{x},\textbf{v},t) d\textbf{v}\\
M = &\int \rho (\textbf{x},t) d\textbf{x}= \int f(\textbf{x},\textbf{v},t) d\textbf{x} d\textbf{v}.
\end{align}
$$


Consider a particle of mass $m$ at position $\textbf{x}$ at time $t$ with velocity $\textbf{v}$. Assume also that some force $\textbf{F}$ acts upon this particle. What, then, happens to this particle at time $t+\Delta t$? In order to answer this question, we proceed using differentiation to obtain the evolution of the probability distribution, i.e.

$$
\begin{align}
\frac{df}{dt} = & \frac{\partial f}{\partial t} + \frac{\partial f}{\partial \textbf{x}} \frac{d \textbf{x}}{dt} + \frac{\partial f}{\partial \textbf{v}} \frac{d \textbf{v}}{dt}\\
\frac{df}{dt} = &\frac{\partial f}{\partial t} + \textbf{v} \cdot \nabla_\textbf{x} f + \frac{\textbf{F}}{m} \cdot \nabla_\textbf{v} f\\
\end{align}
$$

When there are no collisions, this gives the collision-less Boltzmann transport equation,
$$
\frac{df}{dt} = \frac{\partial f}{\partial t} + \textbf{v} \cdot \nabla_\textbf{x} f + \frac{\textbf{F}}{m} \cdot \nabla_\textbf{v} f=0.
$$

To add in collisions, we introduce a collision operator $\Omega(\textbf{x},t)$, yielding
$$
\frac{df}{dt} = \frac{\partial f}{\partial t} + \textbf{v} \cdot \nabla_\textbf{x} f + \frac{\textbf{F}}{m} \cdot \nabla_\textbf{v} f= \Omega (\textbf{x},t).
$$

## Discretization of the Boltzmann Equation

In order to simulate these equations, we must transform them into a set of algebraic equations via discretization. The **Lattice Boltzmann Method** is based on a discretization in which time and space are given, as usual, as a lattice with fixed width steps, and we discretize velocity into a finite number of potential directions. 


The exact form of the discretization of the velocity depends on the space dimensionality. For example, for a two-dimensional setting, a common velocity discretization uses nine possible values in the directions $\textbf{e}_i$ using $i=0,1,2,3,4,5,6,7,8$, with $\textbf{e}_0$ corresponding to the condition of zero velocity,

<img src="https://miro.medium.com/v2/resize:fit:800/format:webp/1*p68lcBLpmVQqT9vreqMTog.png" width=300>

Hence, considering a particular velocity $\textbf{v} = v \textbf{e}_\alpha$, the Boltzmann equation becomes

$$
\frac{\partial f}{\partial t} + v \textbf{e}_\alpha \cdot \nabla_\textbf{x} f + \frac{\textbf{F}}{m} \cdot \nabla_\textbf{v} f= \Omega (\textbf{x},t),
$$

where the term $\textbf{e}_\alpha \cdot \nabla_\textbf{x} f$ corresponds to the directional derivative of $f$ in the direction of $\textbf{e}_\alpha$. 

Assuming that there are no forces acting on the fluid, i.e. $\textbf{F} = 0$, we simplify the equation of motion as

$$
\frac{\partial f}{\partial t} + v \textbf{e}_\alpha \cdot \nabla_\textbf{x} f= \Omega (\textbf{x},t).
$$

## The Collision Term

The collision term $\Omega$ is incredibly complex to model in general. For the purposes of this simple example, we will implemtn a very common assumption known as the Bhatnagar-Gross-Krook (**BGK**) assumption. This establishes that the collision term simply forces the distribution to slowly decay to an equilibrium distribution denoted by $f^{\text{eq}}$. Hence, function $\Omega$ is modelled as

$$
\Omega (\textbf{x},t) = \frac{f^{\text{eq}} (\textbf{x},t) - f(\textbf{x},t)}{\tau}
$$

where $\tau$ is the timescale of which collisions happen. This equilibrium distribution depends on the fluid model’s equation of state. For this example, we will assume an isothermal (constant temperature) fluid, which has a constant sound speed. We define units such that the lattice speed is $c=1$ and use the bulk properties $\rho$ and $\textbf{u}$ (bulk velocity) to obtain equilibrium state

$$
f^{\text{eq}}_j = \omega_j \rho \left( 1 + 3\frac{\textbf{e}_\alpha \cdot \textbf{u}}{c} + \frac{9}{2} \frac{(\textbf{e}_\alpha \cdot \textbf{u})^2}{c^2}- + \frac{3}{2}\frac{\textbf{u} \cdot \textbf{u}}{c^2} \right)
$$

where $\omega_j$ is a weighting function dependent on the exact velocity discretization.

## Lattice-Boltzmann Method Implementation

To implement the LBM method, we consider $\Delta t = \Delta x = 1$ and discretize the Boltzmann equation as

$$
f_j (\textbf{x} + \textbf{e}_\alpha \delta t, t + \delta t) - f_j (\textbf{x}, t) = \Omega_j (\textbf{x}, t)
$$

Then, we separate the solution of this equation into two steps, referred to as *streaming* and *collision* steps. This approach is somewhat analogous to the common splitting mechanism used in solutions to the Navier-Stokes equations.

The streaming step takes care of fluid movement and propagation, i.e. the left side of the equation. As it does not deal with collisions, it can be done entirely separately for each of the discrete velocities, $e_i$. The distribution for a velocity $e_i$ at step $k+1$ for any grid point $\textbf{x}$ is computed by looking up the distribution at $\textbf{x} − \textbf{e}_i \delta t$, which should just be one grid point away.
The streaming step is also where one must deal with some types of boundary conditions. The simplest type of boundary condition is simply the *bounce-back boundary condition*, that fluid incoming at a point hits a wall and bounces back. If the wall is at the edge of the fluid boundary, this is fairly simple. The fluid distribution with a velocity going away from the wall is determined not by some point beyond the wall (there is no fluid there!), but rather by the fluid coming into the wall with the opposite velocity.

After the streaming step, the collision step takes care of the $\Omega$ term. The BGK approximation simply assumes that the net effect of all the collisions is to relax the distribution to some "equilibrium" distribution $f^{\text{eq}}$. Thus, the collision step is simply the computation of this equilibrium, along with the relaxation update in the form

$$
f_j := f_j - \frac{f^{\text{eq}_j - f_j}}{\tau}.
$$

To summarize, an implementation of the LBM method goes through the following steps:

1. **Initialization:** Initialize a velocity distribution, $f_j$; then compute the initial equilibrium distribution, initial density, and initial velocity, as described previously.

2. **Streaming:** Update each $f_j$ by propagating the fluid distribution in the direction $\textbf{e}_\alpha$, applying bounce-back conditions as necessary.

3. **Collision:** Compute the bulk properties density $\rho$ and velocity $\textbf{u}$. Use these to compute the equilibrium distribution $f^\text{eq}_j$. Then, relax the fluid distribution to the equilibrium.

4. **Loop:** Repeat streaming and collision indefinitely to simulate fluid motion.